# Basic Recipe: Static Analysis of a Simple Reform

This is the recipe you should follow first.  Mastering this recipe is a prerequisite for all the other recipes in this cookbook.

When following the recipe as shown below, you will get several instances 
of the same **ignored** error message from deep inside the Pandas
library that is being used by Tax-Calculator.  After conferring with
the Pandas developers, our expectation is these error messages will go
away when we upgrade to Pandas version 0.22.0, which is scheduled to
be released in January 2018, and which fixes a bug in the Pandas
library.  Meanwhile, the error messages are annoying but harmless.

## Imports

In [28]:
from __future__ import print_function  # Necessary only if using Python 2.7.
from taxcalc import *

## Setup

Use publicly-available CPS input file.

NOTE: if you have access to the restricted-use IRS-SOI PUF-based input file
and you have that file (named 'puf.csv') located in the directory
where this script is located, then you can substitute the following
statement for the prior statement:

`recs = Records()`

In [29]:
recs = Records.cps_constructor()

Specify `Calculator` object for static analysis of current-law policy.

In [30]:
pol = Policy()
calc1 = Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


NOTE: `calc1` now contains a PRIVATE COPY of `pol` and a PRIVATE COPY of `recs`,
so we can continue to use `pol` and `recs` in this script without any
concern about side effects from `Calculator` method calls on `calc1`.

Calculate aggregate current-law income tax liabilities for 2018.

In [31]:
calc1.advance_to_year(2018)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

Read JSON reform file and use (the default) static analysis assumptions.

*Requires `reformA.json` to have been downloaded and put in `ingredients` folder.*

In [32]:
reform_filename = './ingredients/reformA.json'
params = Calculator.read_json_param_objects(reform=reform_filename,
                                            assump=None)

Print reform documentation.

In [33]:
print(Calculator.reform_documentation(params))

REFORM DOCUMENTATION
Baseline Growth-Difference Assumption Values by Year:
none: using default baseline growth assumptions
Policy Reform Parameter Values by Year:
2018:
 _II_rt5 : 0.35
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 5
  desc: The third highest tax rate, applied to the portion of taxable income
        below tax bracket 5 and above tax bracket 4.
  baseline_value: 0.33
 _II_rt6 : 0.37
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 6
  desc: The second higher tax rate, applied to the portion of taxable income
        below tax bracket 6 and above tax bracket 5.
  baseline_value: 0.35
 _II_rt7 : 0.42
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 7
  desc: The tax rate applied to the portion of taxable income below tax
        bracket 7 and above tax bracket 6.
  baseline_value: 0.396
 _PT_rt5 : 0.35
  name: Pass-through income tax rate 5
  desc: The third highest tax rate, applied to the portion of income from

Implement reform and check for reform error messages.

In [34]:
pol.implement_reform(params['policy'])
if pol.reform_errors:
    print(pol.reform_errors)
    exit(1)

## Calculate

Specify Calculator object for static analysis of reform policy.

In [35]:
calc2 = Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


Calculate reform income tax liabilities for 2018.

In [36]:
calc2.advance_to_year(2018)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')

## Results

Print total revenue estimates for 2018.

*Estimates in billons of dollars rounded to nearest hundredth of a billion.*

In [37]:
print('2018_CLP_itax_rev($B)= {:.2f}'.format(itax_rev1 * 1e-9))
print('2018_REF_itax_rev($B)= {:.2f}'.format(itax_rev2 * 1e-9))

2018_CLP_itax_rev($B)= 1287.51
2018_REF_itax_rev($B)= 1234.66


Generate several other standard results tables.

In [38]:
# Aggregate diagnostic tables for 2018.
clp_diagnostic_table = calc1.diagnostic_table(1)
ref_diagnostic_table = calc2.diagnostic_table(1)

# Income-tax distribution for 2018 with CLP and REF results side-by-side.
dist_table1, dist_table2 = calc1.distribution_tables(calc2)
assert isinstance(dist_table1, pd.DataFrame)
assert isinstance(dist_table2, pd.DataFrame)
dist_extract = pd.DataFrame()
dist_extract['funits(#m)'] = dist_table1['s006'] * 1e-6
dist_extract['itax1($b)'] = dist_table1['iitax'] * 1e-9
dist_extract['itax2($b)'] = dist_table2['iitax'] * 1e-9
dist_extract['aftertax_inc1($b)'] = dist_table1['aftertax_income'] * 1e-9
dist_extract['aftertax_inc2($b)'] = dist_table2['aftertax_income'] * 1e-9

# Income-tax difference table by expanded-income decile for 2018.
diff_table = calc1.difference_table(calc2, tax_to_diff='iitax')
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tot_change', 'mean',
                'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'agg_diff($b)', 'mean_diff($)',
                'aftertaxinc_diff(%)']
scaling_factors = [1e-6, 1e-9, 1e0, 1e0, 1e0]
for dname, ename, sfactor in zip(dif_colnames, ext_colnames, scaling_factors):
    diff_extract[ename] = diff_table[dname] * sfactor

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


Generate a decile graph.

In [39]:
calc1.decile_graph(calc2)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


Figure(id='56a5f96b-2e85-4e4f-967b-296ccfbd41a6', ...)

## Print tables

CLP diagnostic table for 2018.

In [40]:
print(clp_diagnostic_table)

                                     2018
Returns (#m)                       169.89
AGI ($b)                        10,503.83
Itemizers (#m)                      67.99
Itemized Deduction ($b)          1,500.94
Standard Deduction Filers (#m)      83.86
Standard Deduction ($b)            771.37
Personal Exemption ($b)          1,222.60
Taxable Income ($b)              7,421.92
Regular Tax ($b)                 1,369.71
AMT Income ($b)                  9,664.54
AMT Liability ($b)                  18.02
AMT Filers (#m)                      4.22
Tax before Credits ($b)          1,387.73
Refundable Credits ($b)             76.61
Nonrefundable Credits ($b)          32.93
Reform Surtaxes ($b)                 0.00
Other Taxes ($b)                     9.32
Ind Income Tax ($b)              1,287.51
Payroll Taxes ($b)               1,233.56
Combined Liability ($b)          2,521.07
With Income Tax <= 0 (#m)           72.00
With Combined Tax <= 0 (#m)         48.60


REF diagnostic table for 2018.

In [41]:
print(ref_diagnostic_table)

                                     2018
Returns (#m)                       169.89
AGI ($b)                        10,503.83
Itemizers (#m)                      34.58
Itemized Deduction ($b)          1,010.91
Standard Deduction Filers (#m)     117.28
Standard Deduction ($b)          1,958.91
Personal Exemption ($b)          1,222.60
Taxable Income ($b)              6,993.08
Regular Tax ($b)                 1,317.86
AMT Income ($b)                  9,945.40
AMT Liability ($b)                  16.87
AMT Filers (#m)                      4.40
Tax before Credits ($b)          1,334.73
Refundable Credits ($b)             81.41
Nonrefundable Credits ($b)          27.99
Reform Surtaxes ($b)                 0.00
Other Taxes ($b)                     9.32
Ind Income Tax ($b)              1,234.66
Payroll Taxes ($b)               1,233.56
Combined Liability ($b)          2,468.22
With Income Tax <= 0 (#m)           79.01
With Combined Tax <= 0 (#m)         49.54


Extract of 2018 distribution tables by baseline expanded-income decile.

*Note: deciles are numbered 0-9 with top decile divided into bottom 5%, 
next 4%, and top 1%, in the lines numbered 11-13, respectively.*

In [42]:
print(dist_extract)

    funits(#m)  itax1($b)  itax2($b)  aftertax_inc1($b)  aftertax_inc2($b)
0        16.99      -1.65      -1.65              -5.30              -5.31
1        16.99     -12.65     -12.82             168.67             168.85
2        16.99     -15.20     -17.23             287.66             289.69
3        16.99     -12.74     -17.18             402.40             406.84
4        16.99       2.69      -4.50             520.45             527.65
5        16.99      27.14      17.77             665.73             675.10
6        16.99      63.42      50.65             861.52             874.29
7        16.99     119.61     107.46           1,153.42           1,165.57
8        16.99     212.26     201.22           1,629.28           1,640.32
9        16.99     904.62     910.94           3,599.33           3,593.01
10      169.89   1,287.51   1,234.66           9,283.16           9,336.02
11        8.49     200.80     197.05           1,150.22           1,153.97
12        6.79     318.15

Extract of 2018 income-tax difference table by expanded-income decile.

In [43]:
print(diff_extract)

    funits(#m)  agg_diff($b)  mean_diff($)  aftertaxinc_diff(%)
0        16.99          0.00          0.27                 0.09
1        16.99         -0.17        -10.24                 0.10
2        16.99         -2.03       -119.71                 0.71
3        16.99         -4.43       -260.96                 1.10
4        16.99         -7.19       -423.46                 1.38
5        16.99         -9.37       -551.70                 1.41
6        16.99        -12.77       -751.77                 1.48
7        16.99        -12.15       -715.03                 1.05
8        16.99        -11.05       -650.18                 0.68
9        16.99          6.32        371.95                -0.18
10      169.89        -52.85       -311.09                 0.57
11        8.49         -3.75       -441.16                 0.33
12        6.79         -0.67        -98.94                 0.05
13        1.70         10.74      6,315.38                -1.01
